In [ ]:
import os 
from glob import glob
import pandas as pd

In [ ]:
# 특정 경로의 파일의 목록을 가져오는 기능 
# os 라이브러리를 이용
os.listdir('./')

In [ ]:
# glob 이용 
# 장점 : 파일의 경로와 파일의 이름을 하나의 리스트로 생성 
#       특정 확장자만 선택해서 리스트로 생성이 가능
json_list = glob("./*.json")

In [ ]:
# json_list를 이용하여 하나의 데이터프레임으로 단순 행 결합

# 빈 데이터프레임을 생성
total_df = pd.DataFrame()

for file_path in json_list:
    # print(file_path)
    df = pd.read_json(file_path)
    # total_df, df를 단순 행결합을 하여 total_df에 대입 
    total_df = pd.concat( [total_df, df], axis=0 )
    # print(df)
    # break
total_df.reset_index(drop=True, inplace=True)

In [ ]:
total_df.info()

In [ ]:
pd.concat(
    [ pd.read_json(file_path) for file_path in json_list[:5] ]
).info()

In [ ]:
# Aspects 의 데이터를 하나로 합치고 새로운 데이터 프레임을 생성 
aspect_df = pd.DataFrame(sum(total_df['Aspects'], []))

In [ ]:
aspect_df.info()

In [ ]:
# 데이터의 분균형 문제 확인 
aspect_df['SentimentPolarity'].value_counts()

In [ ]:
aspect_df.isna().sum()

In [ ]:
# 데이터셋에서 문자열의 좌우의 공백을 제거 
# 모든 컬럼이 Object 형이기 때문에 strip() 바로 사용 가능
aspect_df = aspect_df.map(lambda x : x.strip())

In [ ]:
aspect_df.isin(['']).sum()

In [ ]:
aspect_df['SentimentText'].value_counts()

In [ ]:
before_cnt = len(aspect_df)

aspect_df.drop_duplicates('SentimentText', inplace=True)

after_cnt = len(aspect_df)

print(f"제거가 된 행의 개수 {before_cnt - after_cnt}")

In [ ]:
# 1, 0, -1 의 비율을 확인 
aspect_df['SentimentPolarity'].value_counts()

In [ ]:
# 인덱스를 초기화 
aspect_df.reset_index(drop=True, inplace=True)

In [ ]:
# 토큰화 -> 백터화 
from konlpy.tag import Komoran
from sklearn.feature_extraction.text import TfidfVectorizer

komoran = Komoran()
allow_pos = ['NNP', 'NNG', 'VV', 'VA', 'MAG', 'SL']

def komoran_tokenize(text):
    tokens = []
    for word, pos in komoran.pos(text):
        if (pos in allow_pos) & (len(word) >= 2)  :
            tokens.append(word)
    return tokens

vectorizer = TfidfVectorizer(
    tokenizer= komoran_tokenize, 
    ngram_range=(1, 2), 
    min_df = 3, 
    max_df=0.8, 
    max_features=30000
)

In [ ]:
# 모델 생성 
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

In [ ]:
svc = LinearSVC(random_state=42, class_weight='balanced')

multi_model = MultiOutputClassifier(svc)

pipe = Pipeline(
    [
        ('vector', vectorizer), 
        ('model', multi_model)
    ]
)


In [ ]:
# 계층화 폴드 
from sklearn.model_selection import KFold

skfold = KFold(n_splits=3, shuffle= True, 
                         random_state=42)

In [ ]:
aspect_df.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
aspect_df['Aspect'] = le.fit_transform(aspect_df['Aspect'])
aspect_df['SentimentPolarity'] = aspect_df[
    'SentimentPolarity'].astype('int')

In [ ]:
# 독립 변수 , 종속 변수 생성
X = aspect_df['SentimentText'].values
Y = aspect_df[['Aspect', 'SentimentPolarity']].values

In [ ]:
print(X.shape, Y.shape)

In [ ]:
print(type(Y[0][0]), type(Y[0][1]))

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {
    'model__estimator__C' : [1.0, 2.0]
}
grid = GridSearchCV(
    estimator=pipe, 
    param_grid= params, 
    cv = skfold, 
    scoring="accuracy"
)

In [ ]:
grid.fit(X, Y)

In [108]:
grid.best_score_

np.float64(nan)